# Michelson kernel basics
Welcome, friend! This an ultimate Michelson playground, and in this tutorial, you will learn how to make the most of all available functionality.  
If there are any questions, please ask in our telegram chat https://t.me/baking_bad_chat

In [4]:
PUSH string "Hello, world!"

value,type
"""Hello, world!""",string


Michelson kernel is built on top of a custom interpreter which does not typecheck the whole script before execution but at runtime instead. Also, it allows developer to check the stack state at any time and for arbitrary depth.  
This enables a step-by-step coding approach which can save time while learning language or making a prototype or demo.

In [5]:
DROP

## Context, stack, and notebook cells
When you start a kernel, a new instance of context is initialized. It stores the Michelson stack, stub values for the blockchain-specific instructions (e.g. `BALANCE` or `SENDER`), big map pool, origination index, and some internal variables.  
This context is shared across all cells. Note, that the cell's position doesn't matter, only the execution order.

In [6]:
BALANCE  # balance is initialized with a default value

value,type,name
257000000,mutez,@balance


When you execute a cell, messages can appear in the `stdout` and` stderr` streams, as well as the optional result at the end.  
Let's try to execute a sequence of commands:

In [7]:
PUSH mutez 0 ;
IFCMPEQ { FAIL } { PUSH string "We got money!" }

PUSH: push 0;
COMPARE: pop 0, 257000000; push -1;
EQ: pop -1; push False;
IF: pop False;
  PUSH: push We got money!;

value,type
"""We got money!""",string


When there's more than one command a verbose logging is enabled. It can be disabled which will be shown a bit later.  
If the latest command in the sequences pushes an item onto the stack - it will be returned as a result.

### Inspecting the stack

In [8]:
DEBUG False  # we just have disabled the verbose output

In [9]:
PUSH (list string) { "a" ; "b" ; "c" } ;
PRINT "{0} is on top of the stack, then goes {1}" ;  # still we can printf anything to stdout
CONCAT @abc

PRINT: ['a', 'b', 'c'] is on top of the stack, then goes We got money!;

value,type,name
"""abc""",string,@abc


Time to inspect the stack, there is `DUMP` helper for that. You can use it with or without depth specified (all elements)

In [10]:
DUMP 2

value,type,name
"""abc""",string,@abc
"""We got money!""",string,


In [11]:
DROP_ALL ; DUMP

stack is empty

### Errors
if any instruction in a cell has failed the whole context is rolled back to the previous state. So you don't have to rerun everything from the beginning. Here are a few examples of possible errors:

In [12]:
ADD

MichelsonRuntimeError: got 0 items, requested 2 
at ADD

In [13]:
HELLO

MichelsonParserError: unknown primitive HELLO
at line 1, pos 0

In [14]:
PUSH mutez 1 ; EQ

MichelsonRuntimeError: expected Int, got Mutez
at EQ

In [15]:
DEBUG True  # and we continue to the next topic ^_^

## Blockchain-specific instructions
First of all, there are several instructions that in a real environment push some value from the execution context, as `AMOUNT`, `SENDER`, `SOURCE`, `BALANCE`, etc. Here we are detached from any particular chain, but you have an opportunity to patch these values:

In [16]:
PATCH AMOUNT 100500 ; AMOUNT

PATCH: set AMOUNT=100500;
AMOUNT: push 100500;

value,type,name
100500,mutez,@amount


In [17]:
PATCH AMOUNT ; AMOUNT  # still have default value 0

PATCH: unset AMOUNT;
AMOUNT: push 0;

value,type,name
0,mutez,@amount


### Internal operations
Despite internal operations will never apply, Michelson kernel tries to emulate the standard behavior as closely as possible.

In [18]:
UNIT;                    # starting storage for contract
AMOUNT;                  # Push the starting balance
NONE key_hash;           # No delegate
CREATE_CONTRACT          # Create the contract
{ parameter unit ;
  storage unit ;
  code { FAIL } };

UNIT: push Unit;
AMOUNT: push 0;
NONE: push None;
CREATE_CONTRACT: pop None, 0, Unit; set BALANCE=257000000; push KT1Mjjcb6tmSsLm7Cb3DSQszePjfchPM4Uxm; push <originate KT1Mjjcb6tmSsLm7Cb3DSQszePjfchPM4Uxm>;

value,type
amount: '0' code: code { { UNIT ; FAILWITH } } kind: origination storage: Unit target: KT1Mjjcb6tmSsLm7Cb3DSQszePjfchPM4Uxm,operation


### Script sections
`parameter`, `storage`, and `code` instructions are supported as well, what they do is basically store the argument in the context.

In [19]:
parameter unit ;
storage string ;
code { DROP ; PUSH string "Hey!"; NIL operation ; PAIR }

parameter: set parameter=unit;
storage: set storage=string;
code: set code={ DROP ; PUSH string "Hey!" ; NIL operation ; PAIR };

In order to run this contract with particular parameters and initial storage you can use `RUN` helper:

In [20]:
RUN %default Unit "hi"  # %default is entrypoint

RUN: use %default; drop all; push (Unit, 'hi');
  DROP: pop (Unit, 'hi');
  PUSH: push Hey!;
  NIL: push [];
  PAIR: pop [], Hey!; push ([], 'Hey!');

value,type
"""Hey!""",string


`RUN` returns storage, big map diff (if applicable), and list of internal operations.  
You can also load contract from file or chain via `INCLUDE` helper:

In [1]:
INCLUDE "KT1VG2WtYdSWz5E7chTeAdDPZNy2MpP8pTfL%mainnet"  # can also be a filename

INCLUDE:
  parameter: set parameter=or (or %fund (pair :initiate %initiate (address %participant) (pair %settings (pair (bytes %hashed_secret) (timestamp %refund_time)) (mutez %payoff))) (bytes :hashed_secret %add)) (or %withdraw (bytes :secret %redeem) (bytes :hashed_secret %refund));
  storage: set storage=pair (big_map bytes (pair (pair %recipients (address %initiator) (address %participant)) (pair %settings (pair (mutez %amount) (timestamp %refund_time)) (mutez %payoff)))) unit;
  code: set code={ NIL @operations operation ; SWAP ; { { DUP ; CAR @% ; DIP { CDR } } ; DIP { { DUP ; CAR @% ; DIP { CDR @% } } } } ; DIP { DUP } ; IF_LEFT { IF_LEFT { { { DUP ; CAR @% ; DIP { CDR @% } } } ; DUP ; CONTRACT @participant unit ; { IF_NONE { { UNIT ; FAILWITH } } {} } ; DROP ; SWAP ; { { DUP ; CAR ; DIP { CDR @% } } ; { DUP ; CAR @% ; DIP { CDR @% } } } ; DUP ; SIZE ; PUSH nat 32 ; { { COMPARE ; EQ } ; IF {} { { UNIT ; FAILWITH } } } ; DIP { DIP { DUP } ; SWAP ; AMOUNT @amount ; SUB ; SENDER ;

### Step by step debugging
In case you don't want to execute the whole `code`, you can mark the beginning of the contract by calling `BEGIN` (with the same arguments as `RUN`) and in the end call `COMMIT`:

In [2]:
BEGIN %refund 0xdeadbeef (Pair {} Unit)

value,type
Pair 0xdeadbeef (Pair -1 Unit),pair (bytes :hashed_secret %refund) (pair (big_map bytes (pair (pair %recipients (address %initiator) (address %participant)) (pair %settings (pair (mutez %amount) (timestamp %refund_time)) (mutez %payoff)))) unit)


In [3]:
CDR ; NIL operation ; PAIR ; COMMIT

CDR: pop (b'\xde\xad\xbe\xef', (-1, Unit)); push (-1, Unit);
NIL: push [];
PAIR: pop [], (-1, Unit); push ([], (-1, Unit));
COMMIT:

value        type
-----------  ---------------------------------------------------------------------------------------------------
Pair 0 Unit  pair
               (big_map bytes
                        (pair (pair %recipients (address %initiator) (address %participant))
                              (pair %settings (pair (mutez %amount) (timestamp %refund_time)) (mutez %payoff))))
               unit
  big_map  action    key    value
---------  --------  -----  --------------------------------------------------------------------------------------
        0  alloc     bytes  pair (pair %recipients (address %initiator) (address %participant))
                                  (pair %settings (pair (mutez %amount) (timestamp %refund_time)) (mutez %payoff))

## Big maps
In the previous example, you might notice that we initialize the storage as an empty map, then it is displayed on the stack as -1, and as a result, it becomes 0.  
This is roughly how big map works in the real world: first, a temporary container is created, and if at the end of the contract execution it's still in use - a new big map is allocated. Basically big map is an integer pointer to a hashtable somewhere in the context.  
In our playground in order to check the big map state, you need to call `BIG_MAP_DIFF` helper.

In [4]:
EMPTY_BIG_MAP string string

value,type
-2,big_map string string


In [5]:
PUSH string "two";
SOME;
PUSH string "one";
UPDATE

PUSH: push two;
SOME: pop two; push ('two',);
PUSH: push one;
UPDATE: pop one, ('two',), -2; push -2;

value,type
-2,big_map string string


In [7]:
BIG_MAP_DIFF  # works if the top item contains big maps

big_map,action,key,value
1,alloc,string,string
1,update,"""one""","""two"""
